In [1]:
path = "MyDrive/DSAIL"
def mount_drive():
  from google.colab import drive
  mount_location = '/content/drive'
  drive.mount(mount_location,force_remount=True)
  return mount_location
import os, sys
mount_location = mount_drive()
print("your google drive is mounted at: ", mount_location)
path = os.path.join(mount_location,path) # "/content/drive/MyDrive/"
if os.path.exists(path):
  print("Path exists\n\t", path)
  sys.path.append(path)
  print("Path added (file under this path automatically identified)\n\t", path)
  os.chdir(path)
else :
  raise ValueError("Path does not exist. Set proper path \n", path)

Mounted at /content/drive
your google drive is mounted at:  /content/drive
Path exists
	 /content/drive/MyDrive/DSAIL
Path added (file under this path automatically identified)
	 /content/drive/MyDrive/DSAIL


In [3]:
import torch
from torch import nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from math import sqrt
from types import SimpleNamespace
import torch.nn.init as init

device = 'cpu' if not torch.cuda.is_available() else 'cuda'
device

test = np.load("continous_test.npy")
train = np.load("continous_train.npy")

# implicit으로 바꾸기
train = np.where(train > 0, 1, 0)
test = np.where(test > 0, 1, 0)

In [15]:
import torch
from torch.utils.data import DataLoader, Dataset
import random

train_tensor = torch.tensor(train, dtype=torch.float32)
test_tensor = torch.tensor(test, dtype=torch.float32)

class UserItemDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        rating = self.data[idx]
        return idx, rating

train_dataset = UserItemDataset(train_tensor)
test_dataset = UserItemDataset(test_tensor)


In [17]:
user, rating = train_dataset[0]
print(user, rating)

0 tensor([1., 1., 1.,  ..., 0., 0., 0.])


In [20]:
train_loader = DataLoader(train_dataset, batch_size= 3, shuffle=True)
i = 0
for label, data in train_loader:
  if i > 1:
    break
  i += 1
  print("label", label)
  print("data", data)


label tensor([ 42, 250, 652])
data tensor([[0., 0., 1.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [1., 1., 0.,  ..., 0., 0., 0.]])
label tensor([103, 664,  32])
data tensor([[0., 0., 1.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])


In [ ]:
config = SimpleNamespace(
    seed = 1,
    batch_size = 256,
    test_batch_size = 16,
    lr = 1e-3,
    num_user = train.shape[0],
    num_item = train.shape[1],
    epoch = 100,
    r = 64, # hidden dimension
    m = 0.5,
    lambC = 10,
    batch_size = 64,
)

train_loader = DataLoader(train_dataset, batch_size= config.batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size= config.batch_size, shuffle=False)

print("train", train_dataset.shape)
print("test", test_dataset.shape)
print("train_loader_length", len(train_loader))

class CML(nn.Module):
  def __init__(self, config, train = train, train_loader = train_loader, test_loader = test_loader):
    super(CML, self).__init__()
    '''
    CML
    user - item 관계를 당김으로써 나머지 user - user / item- item 간의 거리도 가까워짐

    User / Item representation을 완성하는게 목표 (r 차원)

    '''
    self.config = config
    self.num_user, self.num_item = config.num_user, config.num_item
    self.r = config.r

    self.w = torch.randn(self.num_user, self.num_item) # ranking weight
    self.m = config.m # margin > 0
    self.U = 10 # (10 ~ 20)

    #########################################
    # 원래는 item description 이용한 feature extract 부분이 포함되어야 함 (패쓰)
    # self.f = nn.Linear(in_feature, out_feature)
    #########################################

    # Parameters
    self.u = nn.Parameter(torch.randn(self.num_user, self.r) / self.num_user)
    self.v = nn.Parameter(torch.randn(self.num_item, self.r) / self.num_item)

    self.optimizer = optim.Adam(self.parameters(), lr = config.lr)

    self.lambC = config.lambC

    self.train_loader = train_loader
    self.test_loader = test_loader

    self.positive_pairs = {}
    # user와 positive item
    for u in range(self.num_user):
      self.positive_pairs[u] = []
      for j in range(self.num_item):
        if train[u][j] != 0
          self.positive_pairs[u].append(j)

  def d(self, i, j):
    squared_diff = (self.u[i] - self.v[j]) ** 2
    distance = torch.sqrt(torch.sum(squared_diff))

    return distance

  def distatnce_pos_neg(self, i, j, k):
    '''
    distance
    (i, j): positive
    (i, k): negative
    m : margin
    '''
    distatnce_loss = torch.relu(self.m + self.d(i, j) - self.d(i, k))

    return distatnce_loss
  def update_rank(self):
    # self.w_ij
    for i in range(self.num_user):
      for j in self.positive_pairs[i]:
        d_ij = self.d(i, j)
        negative_items = [item for item in range(self.num_item) if item not in self.positive_pairs[i]]
        random_items = random.sample(negative_items, k= self.U)
        k = 0 # num of impostor
        for random in random_items:
          if self.d(i, random) < d_ij:
            k += 1
        self.w[i][j] = k / self.U * self.num_item

# 원래대로 라면 Lf도 있어야 함 rnlcksgek,,

  def covariance_reg(self):
    '''
    기존의 l2 norm은 gradient 가야돼서 이렇게 하기
    '''
    C = None
    return C

  def forward(self, batch_user):
    # 원래는 batch 단위로 넣으려고 했는데,,
    loss_m = 0
    for i in batch_user:
      for j in self.positive_pairs[i]:
        negative_items = [item for item in range(self.num_item) if item not in self.positive_pairs[i]]
        for k in negative_items:
          loss_m += self.distatnce_pos_neg(i, j, k)

    loss = self.covariance_reg + loss_m * self.lambC
    return loss


  def fit(self):
    train_loss = []
    for epoch in range(self.epochs):
      batch_loss = 0
      for user, ratings in self.train_loader:
        self.optimizer.zero_grad()
        loss = self.forward(user)
        batch_loss += (loss.item())
        loss.backward()
        self.optimizer.step()
        train_loss.append(loss.item())
      print(f"Train loss{ batch_loss / len(self.train_loader)}")
    return train_loss

## 미완성
언젠가.....